In [1]:
# 尝试使用脚本，操作Google网页

In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from selenium.common.exceptions import TimeoutException

In [33]:
driver = webdriver.Chrome()
driver.get("https://earth.google.com/")

lat = 30.14682631
lon = 114.3440979

In [34]:
# 点击包装的 flt-semantics 元素以触发下拉框（发现首次刷新界面不会出现此元素）
try:
    search_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'flt-semantics[role="button"]'))
    )
    search_button.click()
    print("button clicked.")
except TimeoutException:
    # 使用 JavaScript 点击搜索框可能出现的位置
    driver.execute_script("document.querySelector('flt-semantics-placeholder').click();")
    print("click placeholder.")
    search_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'flt-semantics[role="button"]'))
    )
    search_button.click()
    print("button clicked.")

# 循环判断下拉框是否出现，时间间隔为2秒
dropdown_visible = False
while not dropdown_visible:
    try:
        dropdown = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'flt-semantics[aria-label="搜索建议"]'))
        )
        dropdown_visible = True  # 如果找到了下拉框元素，退出循环
    except TimeoutException:
        print("下拉框未出现，等待2秒再尝试...")
        time.sleep(2)  # 等待2秒再检查

# 找到并点击实际的输入框
real_input = WebDriverWait(driver, 30).until(
#     EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[aria-label="搜索 Google 地球"]'))
    EC.element_to_be_clickable((By.TAG_NAME, 'input'))
)
# real_input.click() # 被遮挡住了
driver.execute_script("arguments[0].click();", real_input)

# 输入经纬度并触发搜索
real_input.send_keys(str(lat) + ","+ str(lon))
time.sleep(1)
real_input.send_keys(Keys.ENTER)
print(f"search finish. ({lat}, {lon})")


click placeholder.
button clicked.
search finish. (30.14682631, 114.3440979)


In [35]:
# 等待地图加载完成
try:
    element_found = False

    while not element_found:
        # 查找所有 flt-semantics 元素
        elements = driver.find_elements(By.TAG_NAME, 'flt-semantics')

        for element in elements:
            aria_label = element.get_attribute('aria-label')
            if aria_label and "google_logo" in aria_label:
                print(f"找到包含 'google_logo' 的元素，aria-label: {aria_label}")
                if "100%" in aria_label:
                    print("地图加载结束。")
                    element_found = True
                    break

        if not element_found:
            print("等待地图加载...")
            time.sleep(2)

except Exception as e:
    print(f"发生错误: {e}")

找到包含 'google_logo' 的元素，aria-label: 加载当前卫星图像的进度
google_logo
100%
2022/7/24
30°08'48"N 114°20'40"E
40 米 100%
地图加载结束。


In [36]:
# 1. 关闭搜索后弹出的提示框
try:
    close_alert = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//flt-semantics[text()="关闭"]'))
    )
#     flt_semantics_element.click() # 被遮挡住了
    driver.execute_script("arguments[0].click();", close_alert)
    print("元素1已点击")

except TimeoutException:
    print("未能找到具有特定文本的元素1或元素1不可点击")
    
# 2. 关闭菜单栏
try:
    flt_semantics_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//flt-semantics[@aria-label="隐藏菜单栏"]'))
    )
    
    # 点击元素
    flt_semantics_element.click()
    print("元素2已点击")

except TimeoutException:
    print("未能找到具有特定 aria-label 的元素2或元素2不可点击")
    
# 3. 关闭（左侧）面板
try:
    close_board = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//flt-semantics[text()="收起面板"]'))
    )
#     flt_semantics_element.click() # 被遮挡住了
    driver.execute_script("arguments[0].click();", close_board)
    print("元素3已点击")

except TimeoutException:
    print("未能找到具有特定文本的元素3或元素3不可点击")
    
# 4. 发送 F11 键，网页全屏
driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.F11)

元素1已点击
元素2已点击
元素3已点击


In [43]:
'''5. 设置顶部栏不可见（不好找，有多层。可放弃，截图是计算后忽略顶部栏高度和底部栏高度即可。）'''
try:
    # 查找子元素 aria-label 为 "显示菜单栏" 的 flt-semantics 元素
    sub_element = driver.find_element(By.XPATH, '//flt-semantics[@aria-label="显示菜单栏"]')
    # 然后定位到第二层的祖先： flt-semantics 元素
    grandpa_element = sub_element.find_element(By.XPATH, './ancestor::flt-semantics[1]')
    # 打印找到的父元素
    print("顶部菜单栏：", grandpa_element.get_attribute('id'))

    # 如果需要，可以对这个父元素进行其他操作，比如设置不可见
    driver.execute_script("arguments[0].style.visibility='hidden';", grandpa_element)
except Exception as e:
    print("顶部菜单获取错误：", e)


顶部菜单栏： flt-semantic-node-33


In [44]:
# 完成后关闭 WebDriver
driver.quit()